In [43]:
from pathlib import Path
import pandas as pd

In [44]:
data = Path('Resources/Change_in_Mean_Sea_Levels.csv')
df = pd.read_csv(data)
df.head()

,ObjectId,Country,ISO2,ISO3,Indicator,Unit,Source,CTS_Code,CTS_Name,CTS_Full_Descriptor,Measure,Date,Value
0,1,World,NaN,WLD,Change in mean sea level: Sea level: TOPEX.Pos...,Millimeters,National Oceanic and Atmospheric Administratio...,ECCL,Change in Mean Sea Level,"Environment, Climate Change, Climate Indicator...",Andaman Sea,D12/17/1992,-10.34
1,2,World,NaN,WLD,Change in mean sea level: Sea level: TOPEX.Pos...,Millimeters,National Oceanic and Atmospheric Administratio...,ECCL,Change in Mean Sea Level,"Environment, Climate Change, Climate Indicator...",Arabian Sea,D12/17/1992,-18.46
2,3,World,NaN,WLD,Change in mean sea level: Sea level: TOPEX.Pos...,Millimeters,National Oceanic and Atmospheric Administratio...,ECCL,Change in Mean Sea Level,"Environment, Climate Change, Climate Indicator...",Atlantic Ocean,D12/17/1992,-15.41
3,4,World,NaN,WLD,Change in mean sea level: Sea level: TOPEX.Pos...,Millimeters,National Oceanic and Atmospheric Administratio...,ECCL,Change in Mean Sea Level,"Environment, Climate Change, Climate Indicator...",Baltic Sea,D12/17/1992,196.85
4,5,World,NaN,WLD,Change in mean sea level: Sea level: TOPEX.Pos...,Millimeters,National Oceanic and Atmospheric Administratio...,ECCL,Change in Mean Sea Level,"Environment, Climate Change, Climate Indicator...",Bay Bengal,D12/17/1992,3.27


In [45]:
df.nunique()


ObjectId               35604
Country                    1
ISO2                       0
ISO3                       1
Indicator                  5
Unit                       1
Source                     1
CTS_Code                   1
CTS_Name                   1
CTS_Full_Descriptor        1
Measure                   25
Date                    3653
Value                  16831
dtype: int64

In [46]:
df_2 = df[["Measure", "Date", "Value"]]
df_2.head()

,Measure,Date,Value
0,Andaman Sea,D12/17/1992,-10.34
1,Arabian Sea,D12/17/1992,-18.46
2,Atlantic Ocean,D12/17/1992,-15.41
3,Baltic Sea,D12/17/1992,196.85
4,Bay Bengal,D12/17/1992,3.27


In [47]:
from datetime import date
from dateutil.relativedelta import relativedelta
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [48]:
df_2['Measure'].value_counts()

World             1426
Tropics           1426
Nino              1426
Indian Ocean      1426
Southern Ocean    1426
Pacific Ocean     1426
Atlantic Ocean    1426
Sea Okhotsk       1425
Caribbean Sea     1425
Bering Sea        1425
Gulf Mexico       1425
North Pacific     1425
Indonesian        1425
Arabian Sea       1425
South China       1425
North Atlantic    1425
North Sea         1424
Bay Bengal        1424
Sea Japan         1424
Baltic Sea        1424
Mediterranean     1424
Andaman Sea       1423
Yellow Sea        1422
Adriatic Sea      1418
Persian Gulf      1414
Name: Measure, dtype: int64

In [49]:
measure_dummies = pd.get_dummies(df_2['Measure'])
measure_dummies.head()

,Adriatic Sea,Andaman Sea,Arabian Sea,Atlantic Ocean,Baltic Sea,Bay Bengal,Bering Sea,Caribbean Sea,Gulf Mexico,Indian Ocean,...,North Sea,Pacific Ocean,Persian Gulf,Sea Japan,Sea Okhotsk,South China,Southern Ocean,Tropics,World,Yellow Sea
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
#Merge the dataframes
df_3 = df_2.merge(measure_dummies, left_index=True, right_index=True)
df_3.head()

,Measure,Date,Value,Adriatic Sea,Andaman Sea,Arabian Sea,Atlantic Ocean,Baltic Sea,Bay Bengal,Bering Sea,...,North Sea,Pacific Ocean,Persian Gulf,Sea Japan,Sea Okhotsk,South China,Southern Ocean,Tropics,World,Yellow Sea
0,Andaman Sea,D12/17/1992,-10.34,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Arabian Sea,D12/17/1992,-18.46,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Atlantic Ocean,D12/17/1992,-15.41,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Baltic Sea,D12/17/1992,196.85,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bay Bengal,D12/17/1992,3.27,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:

df_3.drop(columns="Measure", inplace=True)
df_3.head()

,Date,Value,Adriatic Sea,Andaman Sea,Arabian Sea,Atlantic Ocean,Baltic Sea,Bay Bengal,Bering Sea,Caribbean Sea,...,North Sea,Pacific Ocean,Persian Gulf,Sea Japan,Sea Okhotsk,South China,Southern Ocean,Tropics,World,Yellow Sea
0,D12/17/1992,-10.34,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,D12/17/1992,-18.46,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,D12/17/1992,-15.41,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,D12/17/1992,196.85,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,D12/17/1992,3.27,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
df_3["Date"] = df_3["Date"].map(lambda x: x.lstrip('D'))
df_3.head()

,Date,Value,Adriatic Sea,Andaman Sea,Arabian Sea,Atlantic Ocean,Baltic Sea,Bay Bengal,Bering Sea,Caribbean Sea,...,North Sea,Pacific Ocean,Persian Gulf,Sea Japan,Sea Okhotsk,South China,Southern Ocean,Tropics,World,Yellow Sea
0,12/17/1992,-10.34,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,12/17/1992,-18.46,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12/17/1992,-15.41,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,12/17/1992,196.85,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12/17/1992,3.27,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
df_3['Date'] = pd.to_datetime(df_3['Date'])

In [56]:
df_3.head()

,Date,Value,Adriatic Sea,Andaman Sea,Arabian Sea,Atlantic Ocean,Baltic Sea,Bay Bengal,Bering Sea,Caribbean Sea,...,North Sea,Pacific Ocean,Persian Gulf,Sea Japan,Sea Okhotsk,South China,Southern Ocean,Tropics,World,Yellow Sea
0,1992-12-17,-10.34,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1992-12-17,-18.46,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1992-12-17,-15.41,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1992-12-17,196.85,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1992-12-17,3.27,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
df_3 = df_3.set_index('Date')

In [67]:
df_3.head()

,Value,Adriatic Sea,Andaman Sea,Arabian Sea,Atlantic Ocean,Baltic Sea,Bay Bengal,Bering Sea,Caribbean Sea,Gulf Mexico,...,Pacific Ocean,Persian Gulf,Sea Japan,Sea Okhotsk,South China,Southern Ocean,Tropics,World,Yellow Sea,Measure
Date,,,,,,,,,,,,,,,,,,,,,
1992-12-17,-10.34,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1992-12-17,-18.46,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1992-12-17,-15.41,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1992-12-17,196.85,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1992-12-17,3.27,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [73]:
y = df_3["Value"]
X = df_3.drop(columns="Value")

In [76]:
y.value_counts()

-2.23      12
 21.57     12
 32.76     11
 24.29     11
-16.63     11
           ..
 171.57     1
 100.33     1
 75.57      1
-79.50      1
 55.06      1
Name: Value, Length: 16831, dtype: int64

In [77]:
print(y.min())
print(y.max())

-401.85
504.95


In [84]:
bins = [-402, -399, -299, -199, -99, 0, 99, 199, 299, 399, 499, 504.95]

group_labels = ["-400 and below", "-399 to -300", "-299 to -200", "-199 to -100", "-99 to 0", "1 to 100", "101 to 200", "201 to 300", "301 to 400", "401 to 500", "501 and up"]


In [85]:
pd.cut(df_3["Value"], bins, labels = group_labels).head()

Date
1992-12-17      -99 to 0
1992-12-17      -99 to 0
1992-12-17      -99 to 0
1992-12-17    101 to 200
1992-12-17      1 to 100
Name: Value, dtype: category
Categories (11, object): ['-400 and below' < '-399 to -300' < '-299 to -200' < '-199 to -100' ... '201 to 300' < '301 to 400' < '401 to 500' < '501 and up']

In [87]:
df_3["Value"] = pd.cut(df_3["Value"], bins, labels = group_labels)
df_3.head()

,Value,Adriatic Sea,Andaman Sea,Arabian Sea,Atlantic Ocean,Baltic Sea,Bay Bengal,Bering Sea,Caribbean Sea,Gulf Mexico,...,Persian Gulf,Sea Japan,Sea Okhotsk,South China,Southern Ocean,Tropics,World,Yellow Sea,Measure,Values
Date,,,,,,,,,,,,,,,,,,,,,
1992-12-17,-99 to 0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-99 to 0
1992-12-17,-99 to 0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-99 to 0
1992-12-17,-99 to 0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-99 to 0
1992-12-17,101 to 200,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,101 to 200
1992-12-17,1 to 100,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,1 to 100


In [91]:
df_3.drop(columns="Values", inplace=True)

In [89]:
df_3.drop(columns="Measure", inplace=True)

In [92]:
df_3.head()

,Value,Adriatic Sea,Andaman Sea,Arabian Sea,Atlantic Ocean,Baltic Sea,Bay Bengal,Bering Sea,Caribbean Sea,Gulf Mexico,...,North Sea,Pacific Ocean,Persian Gulf,Sea Japan,Sea Okhotsk,South China,Southern Ocean,Tropics,World,Yellow Sea
Date,,,,,,,,,,,,,,,,,,,,,
1992-12-17,-99 to 0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1992-12-17,-99 to 0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1992-12-17,-99 to 0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1992-12-17,101 to 200,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1992-12-17,1 to 100,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
value_dummies = pd.get_dummies(df_3["Value"])
value_dummies.head()

,-400 and below,-399 to -300,-299 to -200,-199 to -100,-99 to 0,1 to 100,101 to 200,201 to 300,301 to 400,401 to 500,501 and up
Date,,,,,,,,,,,
1992-12-17,0,0,0,0,1,0,0,0,0,0,0
1992-12-17,0,0,0,0,1,0,0,0,0,0,0
1992-12-17,0,0,0,0,1,0,0,0,0,0,0
1992-12-17,0,0,0,0,0,0,1,0,0,0,0
1992-12-17,0,0,0,0,0,1,0,0,0,0,0


In [99]:
df_4 = df_3.merge(value_dummies, left_index=True, right_index= True)
df_4.head()

,Value,Adriatic Sea,Andaman Sea,Arabian Sea,Atlantic Ocean,Baltic Sea,Bay Bengal,Bering Sea,Caribbean Sea,Gulf Mexico,...,-399 to -300,-299 to -200,-199 to -100,-99 to 0,1 to 100,101 to 200,201 to 300,301 to 400,401 to 500,501 and up
Date,,,,,,,,,,,,,,,,,,,,,
1992-12-17,-99 to 0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1992-12-17,-99 to 0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1992-12-17,-99 to 0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1992-12-17,-99 to 0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1992-12-17,-99 to 0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


KeyError: 0

In [ ]:
df_4.set_index[]

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [69]:
knn = KNeighborsClassifier(n_neighbors=3, algorithm='auto', weights='distance')
knn.fit(x_train, y_train)
knn_prediction = knn.predict(x_test)

In [72]:
knn_matrix = metrics.confusion_matrix(knn_prediction, y_test)
print (knn_matrix)

[[10682]]
